In [26]:
#Part 8 - Introduction to Plans

import torch
import torch.nn as nn
import torch.nn.functional as F

import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

In [27]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

In [28]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

In [29]:
plan_double_abs

<Plan plan_double_abs id:40519765517 owner:me>

In [30]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

(Wrapper)>[PointerTensor | me:59563762657 -> device_1:63581880945]
	Tags: input_data 
	Shape: torch.Size([2])

In [31]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


In [32]:
plan_double_abs.build(torch.tensor([1., -2.]))
plan_double_abs.is_built

True

In [33]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:32776794092 -> device_1:40519765517]

In [34]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

(Wrapper)>[PointerTensor | me:42358830107 -> device_1:69639717806]


In [35]:
pointer_to_result.get()

tensor([2., 4.])

In [36]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [37]:
net = Net()
net

<Net Net id:20947563742 owner:me>

In [38]:
net.build(torch.tensor([1., 2.]))

In [39]:
pointer_to_net = net.send(device_1)
pointer_to_net

[PointerPlan | me:41208311581 -> device_1:20947563742]

In [40]:
pointer_to_data = device_1.search('input_data')[0]

In [41]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:61522700205 -> device_1:45317370244]

In [42]:
pointer_to_result.get()

tensor([-0.7565, -0.6336], requires_grad=True)

In [43]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [44]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

In [45]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

tensor([-1.4389, -0.2708], requires_grad=True)

In [46]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

tensor([-1.4389, -0.2708], requires_grad=True)

In [47]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

True

In [48]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

True

In [49]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [50]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

tensor([2, 1])